In [1]:
!apt-get update -y
!apt-get install -y python-xlsxwriter
!pip install openpyxl

Get:1 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Err:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
Get:6 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [186 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic/main i386 Packages [1,328 kB]    
Get:8 http://archive.ubuntu.com/ubuntu bionic/multiverse i386 Packages [177 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic/main amd64 Packages [1,344 kB]   
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main am

     |████████████████████████████████| 242 kB 3.4 MB/s            


In [2]:
import wandb
import pandas as pd
import datetime
import tqdm

username = "memmelma"
project = "final"

filter_name = "cnn"
filter_type = "vo_"
# filter_state = "running"
filter_state = "finished"
# filter_state = ""

filter_metric = "Objective/eval_all"
filter_operation = "min"

metrics = ["Objective/eval_forward", "Objective/eval_left", "Objective/eval_right", "Objective/train",
           "eval_forward_unified_depth/aux_depth_loss", "eval_left_unified_depth/aux_depth_loss", "eval_right_unified_depth/aux_depth_loss", "eval_all_unified_depth/aux_depth_loss", "train_unified_depth/aux_depth_loss"]

# configs = ["config.VO.MODEL.visual_backbone", "config.VO.MODEL.pretrain_backbone",
#            "config.VO.MODEL.cls_action", "config.VO.TRAIN.depth_aux_loss"]
configs = ["config.VO.MODEL.visual_backbone", "config.VO.MODEL.pretrain_backbone", "config.VO.MODEL.train_backbone", "config.VO.MODEL.visual_type",
           "config.VO.MODEL.cls_action"]

step_threshold = 150

out_name = "exp_vo_scr"

In [3]:
# retreive runs

run_ids = []
run_names = []
run_ckpts = []

api = wandb.Api(timeout=45)
runs = api.runs(f"{username}/{project}")

runs_df = pd.DataFrame(columns=["name", "id", "state", *configs, filter_metric, *metrics])

for run in tqdm.tqdm(runs):
    
    if filter_state and run._state != filter_state:
        continue
    
    if filter_type and filter_type not in run.name:
        continue
        
    if filter_name and filter_name not in run.name:
        continue
        
    run_dict = {}
    
    # add base attributes
    run_dict["name"] = run.name.split("[")[0]
    run_dict["id"] = run.id 
    run_dict["state"] = run._state
    
    # add config
    for config in configs:
        # traverse config keys
        run_config_tmp = run.config
        for config_iter in config.split('.'):
            run_config_tmp = run_config_tmp[config_iter]
        run_dict[config] = run_config_tmp
    
    # add filter_operation filter_metric
    run_history = run.history()
    select = run_history["_step"]<step_threshold
    
    if filter_operation == "min":
        filter_metric_idx = run_history[filter_metric][select].argmin()
    elif filter_operation == "max":
        filter_metric_idx = run_history[filter_metric][select].argmax()
    else:
        break
    run_dict["best_run_path"] = run.config["config"]["CHECKPOINT_FOLDER"] + f"/ckpt_epoch_{filter_metric_idx}.pth"
    run_dict["best_run_idx"] = filter_metric_idx
        
    run_dict[filter_metric] = run_history[filter_metric][filter_metric_idx]
    
    # add other metrics corresponding to filter_operation filter_metric
    for metric in metrics:
        # catch models that only evaluate some actions
        if metric in run_history.keys():
            run_dict[metric] = run_history[metric][filter_metric_idx]
        else:
            run_dict[metric] = -1
    
    runs_df = runs_df.append(run_dict, ignore_index=True)
    
runs_df = runs_df.sort_values(by=['name'])
runs_df

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 117/117 [00:13<00:00,  8.99it/s]


,name,id,state,config.VO.MODEL.visual_backbone,config.VO.MODEL.pretrain_backbone,config.VO.MODEL.train_backbone,config.VO.MODEL.visual_type,config.VO.MODEL.cls_action,Objective/eval_all,Objective/eval_forward,Objective/eval_left,Objective/eval_right,Objective/train,eval_forward_unified_depth/aux_depth_loss,eval_left_unified_depth/aux_depth_loss,eval_right_unified_depth/aux_depth_loss,eval_all_unified_depth/aux_depth_loss,train_unified_depth/aux_depth_loss,best_run_idx,best_run_path
12,vo_cnn_act_all,2s9gzbb8,finished,resnet50,None,True,"[rgb, depth, discretized_depth, top_down_view]",False,0.002642,0.000775,0.001396,0.001447,0.004200,0,0,0.0,0,0,107.0,train_log/cnn/vo_cnn_act_all/checkpoints/ckpt_...
0,vo_cnn_act_d,2okvybcf,finished,resnet50,None,True,"[depth, discretized_depth, top_down_view]",False,0.003119,0.000879,0.001619,NaN,0.004536,0,0,NaN,0,0,135.0,train_log/final/cnn/unique/vo_cnn_act_d/checkp...
10,vo_cnn_act_rgb,3dw8mtwm,finished,resnet50,None,True,[rgb],False,0.004504,0.003916,0.001479,0.001544,0.004546,0,0,0.0,0,0,88.0,train_log/final/cnn/unique/vo_cnn_act_rgb/chec...
13,vo_cnn_fwd,2im69pwg,finished,resnet50,None,True,"[rgb, depth, discretized_depth, top_down_view]",False,0.000885,-1.000000,-1.000000,-1.000000,0.000264,-1,-1,-1.0,-1,-1,139.0,train_log/final/cnn/unique/vo_cnn_fwd/checkpoi...
11,vo_cnn_in21k_act_all,2ro97gpw,finished,resnet50,in21k,True,"[rgb, depth, discretized_depth, top_down_view]",False,0.004072,0.001385,0.002117,0.002169,0.004222,0,0,0.0,0,0,75.0,train_log/final/cnn/unique/vo_cnn_in21k_act_al...
1,vo_cnn_in21k_act_d,285ehpzk,finished,resnet50,in21k,True,"[depth, discretized_depth, top_down_view]",False,0.004018,0.001608,0.002113,0.002132,0.002456,0,0,0.0,0,0,124.0,train_log/final/cnn/unique/vo_cnn_in21k_act_d/...
4,vo_cnn_in21k_act_rgb,34zdvybq,finished,resnet50,in21k,True,[rgb],False,0.007949,0.005447,0.003266,0.003414,0.002439,0,0,0.0,0,0,142.0,train_log/final/cnn/unique/vo_cnn_in21k_act_rg...
8,vo_cnn_in21k_fwd,2oy54681,finished,resnet50,in21k,True,"[rgb, depth, discretized_depth, top_down_view]",False,0.001544,-1.000000,-1.000000,-1.000000,0.000417,-1,-1,-1.0,-1,-1,139.0,train_log/final/cnn/unique/vo_cnn_in21k_fwd/ch...
7,vo_cnn_in21k_tl,z0addc6l,finished,resnet50,in21k,True,"[rgb, depth, discretized_depth, top_down_view]",False,0.006623,-1.000000,-1.000000,-1.000000,0.000990,-1,-1,-1.0,-1,-1,72.0,train_log/final/cnn/unique/vo_cnn_in21k_tl/che...
3,vo_cnn_in21k_tlr,2se1k6lf,finished,resnet50,in21k,True,"[rgb, depth, discretized_depth, top_down_view]",False,0.012528,-1.000000,-1.000000,-1.000000,0.000919,-1,-1,-1.0,-1,-1,73.0,train_log/final/cnn/unique/vo_cnn_in21k_tlr/ch...


In [4]:
dt = datetime.datetime.now().strftime("%Y%m%d")
filename = f"{out_name}_{dt}"
filename

'exp_vo_scr_20220513'

In [5]:
# save df as excel file
extension = ".xlsx"
with pd.ExcelWriter(filename+extension) as excel_writer:
    runs_df.to_excel(excel_writer, sheet_name='Sheet1')

In [45]:
# # save df as csv file
# extension = ".csv"
# runs_df.to_csv(filename+extension)